In [38]:
import pandas as pd

In [39]:
df = pd.read_csv("bigout.csv")

In [41]:
sent_stats = df.groupby('IP source').agg(
        sent_packets = ('Total packets', 'sum'),
        sent_bytes = ('Total bytes', 'sum')
).reset_index()

In [42]:
received_stats = df.groupby('IP dest').agg(
        received_packets = ('Total packets', 'sum'),
        received_bytes = ('Total bytes', 'sum')
).reset_index()

In [67]:
# Объединяем статистику по отправленным и полученным данным
result = pd.merge(sent_stats, received_stats, left_on='IP source', right_on='IP dest', how='outer')

# Заполняем NaN значениями для IP, которые не встречаются в одной из категорий
result[['sent_packets', 'sent_bytes', 'received_packets', 'received_bytes']] = result[['sent_packets', 'sent_bytes', 'received_packets', 'received_bytes']].fillna(0)

result['IP source'] = result['IP source'].fillna(result['IP dest'])
# Избавляемся от IP назначения
result = result.drop(columns=['IP dest'])

In [68]:
result['received_packets'] = result['received_packets'].astype('int')
result['received_bytes'] = result['received_bytes'].astype('int')
result['sent_packets'] = result['sent_packets'].astype('int')
result['sent_bytes'] = result['sent_bytes'].astype('int')

In [69]:
result = result.rename(columns={
    'IP source': 'IP address',
    'sent_packets':'Packets sent',
    'sent_bytes':'Bytes sent',
    'received_packets':'Packets received',
    'received_bytes':'Bytes received'
})

In [70]:
result.to_csv(output_file, index=False)

,IP address,Packets sent,Bytes sent,Packets received,Bytes received
0,0.0.0.0,3,1770,0,0
1,107.20.103.220,243,91356,420,273718
2,107.20.158.52,344,26428,364,262722
3,107.20.161.243,27,10525,30,6274
4,107.20.170.67,20,8938,20,2644
...,...,...,...,...,...
1724,98.142.99.171,26,33463,10,1236
1725,98.209.196.102,91,8653,94,6768
1726,98.216.191.85,86,40623,77,5758
1727,99.138.108.122,0,0,3,194
